In [22]:
import pandas as pd
import copy
import time
import numpy as np

In [2]:
def load_data(filepath):
    with open(filepath) as fd:
        headers = [ next(fd) for i in range(6) ]
        df = pd.read_csv(fd, sep='\t', lineterminator='\n')
        df = df.drop([";", "~"], axis = 1)
        return df

In [3]:
def linearize(value, agg=None):
    if agg is None:
        agg = []
    if isinstance(value, (tuple, list)):
        for item in value:
            linearize(item, agg)
    else:
        agg.insert(0,value)
    return agg[:-1], agg[-1]

In [4]:
def unnest(d, keys=[]):
    result = []
    for k, v in d.items():
        if isinstance(v, dict):
            result.extend(unnest(v, keys + [k]))
        else:
            result.append(tuple(keys + [k, v]))
    return result

In [5]:
from collections import defaultdict
from heapq import *

def dijkstra(edges, f, t):
    g = defaultdict(list)
    for l,r,c in edges:
        g[l].append((c,r))
    q, seen, mins = [(0,f,())], set(), {f: 0}
    while q:
        (cost,v1,path) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path = (v1, path)
            if v1 == t: return (cost, path)

            for c, v2 in g.get(v1, ()):
                if v2 in seen: continue
                prev = mins.get(v2, None)
                next = cost + c
                if prev is None or next < prev:
                    mins[v2] = next
                    heappush(q, (next, v2, path))

    return float("inf")

In [20]:
def penalty(graph, source, target, k = 1):
    A = []
    counter = 0
    total_linear = linear_distance[str(source)][target]
    graph_copy = copy.deepcopy(graph)
    tuple_graph = unnest(graph_copy)
    path, _ = linearize(dijkstra(tuple_graph, source, target))
    A.append(path)
    while len(A)!=k:
        for i in range(len(path)-1):
            if linear_distance[str(source)][path[i]] > 0.1*total_linear and linear_distance[str(target)][path[i]] > 0.1*total_linear:
                graph_copy[path[i]][path[i+1]] = round(1.1*graph_copy[path[i]][path[i+1]],2)
        tuple_graph = unnest(graph_copy)
        path, _ = linearize(dijkstra(tuple_graph, source, target))
        counter += 1
        if path not in A:
            A.append(path)
        else:
            if counter > 500:
                del path
                break
            else:
                continue
    return A

In [13]:
if __name__ == "__main__":
    filepath = r'C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Code\Path Choice\Data\final\network_13_07_2021.csv'
    network = pd.read_csv(filepath, usecols = ['From_origi', 'To_destina', 'Total_Minu'])
    network['Total_Minu'] = round(network['Total_Minu']*60,2)
    tt = network.groupby('From_origi')[['To_destina', 'Total_Minu']].apply(lambda x: x.set_index('To_destina').to_dict(orient='index')).to_dict()
    for key in tt:
        for x in tt[key]:
            for y in tt[key][x]:
                tt[key][x] = tt[key][x][y]
    linear_df = pd.read_csv(r"C:\Users\n10405992\OneDrive - Queensland University of Technology\Raghav Malhotra - RM\Code\Path Choice\Data\final\Final linear network.csv").set_index("ID")
    linear_distance = linear_df.to_dict()

0.0


In [31]:
origins = [10423, 10599]
destination = [10415, 10651]
paths = []
for origin in origins:
    for destination in destinations:
        paths = penalty(tt, origin, destination, 15)
        new_paths = []
        for path in paths:
            z = []
            for i in range(len(path)-1):
                z.append(int(str(path[i]) + str(path[i+1])))
            new_paths.append(z)
        output = pd.DataFrame({'ID': np.arange(0,len(new_paths)), 'Path': new_paths})
        output.to_csv("Penalty" + str(origin) + "-" + str(destination) + ".csv")